In [68]:
!pip install lime

DEPRECATION: fairness 0.1.8 has a non-standard dependency specifier BlackBoxAuditing>=0.1.26ggplot. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of fairness or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [69]:
import lime
import pandas as pd
import numpy as np



In [70]:
cusomters = pd.read_csv('https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv')

In [71]:
cusomters.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [72]:
cusomters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [73]:
cusomters.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [74]:
cusomters.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [75]:
cusomters.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [76]:
numeric_features = ['tenure', 'MonthlyCharges' ]

In [77]:
categorical_features = cusomters.nunique()[cusomters.nunique() < 5].keys().tolist()

In [78]:
categorical_features

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [79]:
categorical_features.remove('Churn')

In [80]:
X = cusomters[categorical_features + numeric_features]

In [81]:
X.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,tenure,MonthlyCharges
0,Female,0,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,1,29.85
1,Male,0,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,34,56.95
2,Male,0,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,2,53.85
3,Male,0,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),45,42.30
4,Female,0,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,2,70.70


In [82]:
y = cusomters['Churn']

In [83]:
y

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

In [84]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [85]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])


In [96]:
X = preprocessor.fit_transform(X)

In [97]:
X 

array([[  1.  ,  29.85,   1.  , ...,   0.  ,   1.  ,   0.  ],
       [ 34.  ,  56.95,   0.  , ...,   0.  ,   0.  ,   1.  ],
       [  2.  ,  53.85,   0.  , ...,   0.  ,   0.  ,   1.  ],
       ...,
       [ 11.  ,  29.6 ,   1.  , ...,   0.  ,   1.  ,   0.  ],
       [  4.  ,  74.4 ,   0.  , ...,   0.  ,   0.  ,   1.  ],
       [ 66.  , 105.65,   0.  , ...,   0.  ,   0.  ,   0.  ]])

In [98]:
encode = LabelEncoder()

In [99]:
y = encode.fit_transform(y)

In [100]:
y

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [101]:
from sklearn.model_selection import train_test_split

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [103]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
clf = RandomForestClassifier()

In [105]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [106]:
y_pred = clf.predict(X_test)

In [108]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [109]:
print(accuracy_score(y_test, y_pred))

0.7853492333901193


In [110]:
scaler = StandardScaler()

In [111]:
X_train = scaler.fit_transform(X_train)

In [112]:
X_test = scaler.transform(X_test)

In [113]:
clf = RandomForestClassifier()

In [114]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [115]:
y_pred = clf.predict(X_test)

In [116]:
print(accuracy_score(y_test, y_pred))

0.7864849517319704


In [117]:
from sklearn.model_selection import GridSearchCV

In [118]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20, 25]
}

In [124]:
grid = GridSearchCV(clf, param_grid, cv=5, return_train_score=True, refit=True)

In [125]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15, 20, 25],
                         'n_estimators': [100, 200, 300, 400, 500]},
             return_train_score=True)

In [126]:
grid.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'sqrt',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(),
 'n_jobs': None,
 'param_grid': {'n_estimators': [100, 200, 300, 400, 500],
  'max_depth': [5, 10, 15, 20, 25]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': True,
 'scoring': None,
 'verbose': 0}

In [127]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 500}

In [128]:
accuracy_score(y_test, grid.predict(X_test))

0.8018171493469619

In [129]:
y_pred = grid.predict(X_test)

In [130]:
print(accuracy_score(y_test, y_pred))

0.8018171493469619


In [131]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1294
           1       0.66      0.51      0.58       467

    accuracy                           0.80      1761
   macro avg       0.75      0.71      0.72      1761
weighted avg       0.79      0.80      0.79      1761



In [132]:
print(confusion_matrix(y_test, y_pred))

[[1172  122]
 [ 227  240]]


In [136]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))

0.7098186656252378


In [137]:
results = pd.DataFrame(grid.cv_results_, columns=['mean_test_score', 'std_test_score', 'params'])

In [140]:
results.sort_values(by='mean_test_score', ascending=False)

,mean_test_score,std_test_score,params
9,0.799314,0.013193,"{'max_depth': 10, 'n_estimators': 500}"
7,0.798367,0.012894,"{'max_depth': 10, 'n_estimators': 300}"
8,0.797989,0.013280,"{'max_depth': 10, 'n_estimators': 400}"
6,0.797989,0.013878,"{'max_depth': 10, 'n_estimators': 200}"
2,0.797233,0.010060,"{'max_depth': 5, 'n_estimators': 300}"
4,0.796855,0.010222,"{'max_depth': 5, 'n_estimators': 500}"
0,0.796287,0.008563,"{'max_depth': 5, 'n_estimators': 100}"
5,0.796285,0.010885,"{'max_depth': 10, 'n_estimators': 100}"
3,0.796097,0.011164,"{'max_depth': 5, 'n_estimators': 400}"
1,0.795718,0.010432,"{'max_depth': 5, 'n_estimators': 200}"


In [142]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10)

In [143]:
rf.fit(X_train, y_train).predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [151]:
type(rf)

sklearn.ensemble._forest.RandomForestClassifier

In [158]:
idx = np.random.randint(0, X_test.shape[0])
class_labels = [["churn", "no churn"]]
class_labels[rf.predict([X_test[idx]])[0]]


['churn', 'no churn']


Create explainer

In [159]:
from lime.lime_tabular import LimeTabularExplainer

In [160]:
explainer = LimeTabularExplainer(training_data=X_train, 
                                 kernel_width= None,
                                 feature_names=categorical_features + numeric_features,
                                 class_names=class_labels,
                                 discretize_continuous=False)

In [161]:
idx = np.random.randint(0, X_test.shape[0])